In [168]:
import keras
from   sklearn.model_selection import train_test_split
from   sklearn.datasets import make_classification
from   sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [169]:
X, y = make_classification(
    n_samples=7000, n_features=10, n_informative=2, scale=100, random_state=32,
    n_clusters_per_class=1, hypercube=False, class_sep=10
    )

In [170]:
X = StandardScaler().fit_transform(X)

In [171]:
pd.Series(y).value_counts()

1    3501
0    3499
Name: count, dtype: int64

In [172]:
pd.DataFrame(X).corrwith(pd.Series(y))

0    0.021709
1    0.014055
2    0.983690
3   -0.835300
4   -0.982866
5    0.012760
6   -0.982652
7   -0.013867
8   -0.008053
9    0.007010
dtype: float64

In [173]:
xtrain, xtest, ytrain, ytest = train_test_split(
    X, y, train_size=0.25, random_state=1
)

In [174]:
from functools import partial

def Model():
    Dense = partial(
        keras.layers.Dense,
        activation=keras.activations.elu,
        kernel_initializer=keras.initializers.he_normal
        
    )
    model = keras.models.Sequential([
        Dense(20),
        Dense(10),
        Dense(5),
        Dense(2),
        Dense(1)
    ])
    return model

In [175]:
model = Model()

In [182]:
def power_schedule(eta_0, s=10, c=1):
    def power_fn(epoch):
        return eta_0/(1+( epoch/s ))**c
    return power_fn
power = power_schedule(eta_0=0.01)

In [183]:
lr_power = keras.callbacks.LearningRateScheduler(power)

In [187]:
model.compile(
    optimizer=keras.optimizers.SGD(),
    loss     =keras.losses.binary_crossentropy,
    metrics  =[keras.metrics.binary_accuracy] 
)
hist = model.fit(
    xtrain, ytrain, epochs=10, callbacks=[lr_power]
)

Epoch 1/10
55/55 [==============================] - 1s 1ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0100
Epoch 2/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0091
Epoch 3/10
55/55 [==============================] - 0s 1ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0083
Epoch 4/10
55/55 [==============================] - 0s 1ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0077
Epoch 5/10
55/55 [==============================] - 0s 1ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0071
Epoch 6/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0067
Epoch 7/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0063
Epoch 8/10
55/55 [==============================] - 0s 3ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0059
Epoch 9/10
55/55 [==============================] - 0s 1

In [188]:
power_hist = hist.history

In [189]:
[power(i) for i in hist.epoch]

[0.01,
 0.00909090909090909,
 0.008333333333333333,
 0.007692307692307692,
 0.0071428571428571435,
 0.006666666666666667,
 0.0062499999999999995,
 0.0058823529411764705,
 0.005555555555555556,
 0.005263157894736843]

### expo

In [190]:
def exponencial_scheduling(eta_0=0.1, s=10):
    def exponencial_fn(epoch):
        return eta_0 * (0.1**(epoch/s))
    return exponencial_fn

expo = exponencial_scheduling(eta_0=0.01)

In [191]:
lr_expo = keras.callbacks.LearningRateScheduler(expo)

In [193]:
model.compile(
    optimizer=keras.optimizers.SGD(),
    loss     =keras.losses.binary_crossentropy,
    metrics  =[keras.metrics.binary_accuracy] 
)
hist = model.fit(
    xtrain, ytrain, epochs=10, callbacks=[lr_expo]
)

Epoch 1/10
55/55 [==============================] - 1s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0100
Epoch 2/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0079
Epoch 3/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0063
Epoch 4/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0050
Epoch 5/10
55/55 [==============================] - 0s 1ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0040
Epoch 6/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0032
Epoch 7/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0025
Epoch 8/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949 - lr: 0.0020
Epoch 9/10
55/55 [==============================] - 0s 2

In [194]:
class Exponencial(keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, initial_learning_rate, s=10):
        self.initial_learning_rate = initial_learning_rate
        self.s = s

    def __call__(self, step):
        return self.initial_learning_rate * (0.1**(step/self.s))

In [196]:
model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=Exponencial(0.01)),
    loss     =keras.losses.binary_crossentropy,
    metrics  =[keras.metrics.binary_accuracy] 
)
hist = model.fit(
    xtrain, ytrain, epochs=10
)

Epoch 1/10
55/55 [==============================] - 1s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 2/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 3/10
55/55 [==============================] - 0s 3ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 4/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 5/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 6/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 7/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 8/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 9/10
55/55 [==============================] - 0s 2ms/step - loss: 0.0787 - binary_accuracy: 0.9949
Epoch 10/10
55/55 [==============================] - 0s